# 01. Import & Install Dependencies

In [9]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2                                # Computer Vison Library OpenCV
import numpy as np                        # numpy arrays structure our datasets (numpy instead of tensorflow)
import os                                 # work with operating systems file path
from matplotlib import pyplot as plt      # matplotlib just helps us visualize images a little bit easier
import mediapipe as mp                    # mediapipe (https://google.github.io/mediapipe/)
from math import *

# 02. Using Mediapipe (Google's open-source framework)

In [2]:
mp_face_mesh = mp.solutions.face_mesh              # FaceMesh Model has -- 478 -- Keypoints 
mp_drawing_styles = mp.solutions.drawing_styles    # Drawing styles
mp_drawing = mp.solutions.drawing_utils            # Drawing utilites

## Given the original image and obtaining the values through the model.

In [3]:
def mediapipe_detection(image,model):                 
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                     # Image is no longer writeable
    results = model.process(image)                    # Make prediction
    image.flags.writeable = True                      # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # COLOR CONVERSION RGB 2 BGR
    return image, results

## Storing the keypoints values in the mediapipe into a variable.

In [4]:
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

## Calculating blink rate (Euclidean distance).

In [5]:
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio

# Euclaidean distance
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = sqrt((x1 - x)**2 + (y1 - y)**2)
    if 1> distance:
        return 1
    else:
        return distance


# 03. Extract Position Landmarks (Keypoints Values) 

In [6]:
def extract_landmarks(img, results):
    frame = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    mesh_coords = mesh_coord_detection(img, results)
    
    # extract face 478 keyponits
    face = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks[0] else np.zeros(478*3)
    
    # extract right-eye 16 keypoints
    right_eye_coords = np.array([mesh_coords[p] for p in RIGHT_EYE]).flatten()  
    
    # extract left-eye 16 keypoints
    left_eye_coords = np.array([mesh_coords[p] for p in LEFT_EYE]).flatten()    
    
    # calculating eye ratio
    ratio = np.array(blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)).flatten()  
    
    # return all feature values
    return np.concatenate([face,right_eye_coords,left_eye_coords,ratio])


def mesh_coord_detection(img, results):
    if img is None:
        img_height, img_width= 0,0
    else:
        img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark] if results.multi_face_landmarks[0] else np.zeros(478 *2)

    # returning the list of tuples for each landmarks 
    return mesh_coord

# 04. Setup Folder for Collection

In [7]:
DATA_PATH = os.path.join('ML_Data')         # Path for exported data, numpy arrays 
# C:\Users\Sms Studio\Desktop\University_Research_Projects\LieDetection\Semester Research Program\ML_Data
actions = np.array(['true', 'lie'])       # Actions that we try to detect
                      
true_vdos = 156                        # for taking total true videos.

lie_vdos = 64                        # for taking total lie videos.

sequence_length = 30             # Videos are going to be 30 frames in length

In [8]:
# Create Folders for save keypoints values

for action in actions:
    if action == 'true': 
        for sequence in range(true_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass
    else:
        for sequence in range(lie_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass

#  05. Collect Keypoints Values for Training and Testing

In [ ]:
for action in actions:   # Loop through actions
    if action == 'true':
        for sequence in range(true_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/true/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read() # return status(boolean) , image                               # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/true/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face , left-eye , right-eye , ratio

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path

                    # Keypoints Shape:(1499,) ---> (1434 + 32 + 32 + 1)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values



                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______") 
    else:
        for sequence in range(lie_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/lie/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read()                                # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/lie/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face , left-eye , right-eye keypoints

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path

                    # Keypoints Shape:(1499,) ---> (1434 + 32 + 32 + 1)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values



                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______")

Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing

Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
_____Finish_______
Processing Video Path: Datasets/true/7/7.mp4
Processing Video Path: Datasets/true/7/7.mp4
Processing Video Path: Datasets/true/7/7.mp4
Processing Video Path: Datasets/true/7/7.mp4
Processing Video Path: Datasets/true/7/7.mp4
Processing Video Path: Datasets/true

Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
_____Finish_______
Processing Video Path: Datasets/true/13/13.mp4
Processing Video Path: Datasets/true/13/13.mp4
Processing Video Path: Datasets/true/13/13.mp4
Processing Video Path: Datasets/true/13/13.mp4
Processing Video Path: Datasets/true/13/13.mp4
Processing Video Path: Datasets/true/13/1

Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Vi

Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Vi

...

Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/lie/22/22.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/lie/28/28.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/lie/34/34.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/lie/40/40.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/lie/46/46.mp4
Processing Video Path: Datasets/li

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split # Scikit-learn for our evaluation metrics
from tensorflow.keras.utils import to_categorical   # Converting Data (Encoded)

In [11]:
label_map = {label:num for num, label in enumerate(actions)}  # Actions Label Map  {'true': 0, 'lie': 1} 

In [12]:
sequences, labels = [], []                # like : Features(X) Data , labels(Y) Data
for action in actions:              
    for sequence in range(no_sequences):  
        values = []                       # Save all of diff.frames paricular sequence
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))  # load np array
            values.append(res)                # Append all res 
        sequences.append(values)             # Append all values
        labels.append(label_map[action])   # Append all actions on labels

In [13]:
X_train.shape

NameError: name 'X_train' is not defined

In [14]:
X = np.array(sequences)  # sequences
y = to_categorical(labels).astype(int)  # labels

# Train & Testing Partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

NameError: name 'to_categorical' is not defined

In [15]:
# X.shape : (42, 30, 1499)  -----> ( Sequences(videos) , framers per vdo, keypoints shape [(478 * 3) + (32 * 2) + 1] )
# y.shape : (42, 2) -------->   ( Sequences(videos), labels[actions] )

# X_train.shape : (39, 30, 1499) 
# X_test.shape : (3, 30, 1499)

# y_train.shape : (39, 2) 
# y_test.shape : (3, 2) 

In [16]:
X.shape

(42, 60, 1499)

In [17]:
y.shape

NameError: name 'y' is not defined

In [18]:
X_train.shape

NameError: name 'X_train' is not defined

In [19]:
X_test.shape

NameError: name 'X_test' is not defined

In [20]:
y_train.shape

NameError: name 'y_train' is not defined

In [21]:
y_test.shape

NameError: name 'y_test' is not defined

# 7. Build and Train LSTM Neural Network

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1499)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
actions.shape[0]

2

In [26]:
res = [.7, 0.2, 0.1]

In [27]:
actions[np.argmax(res)]

'true'

In [28]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=50, callbacks=[tb_callback])

NameError: name 'X_train' is not defined

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            400384    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 554,914
Trainable params: 554,914
Non-trai

# 8. Make Predictions

In [31]:
res = model.predict(X_test)

NameError: name 'X_test' is not defined

In [32]:
res

[0.7, 0.2, 0.1]

In [33]:
np.sum(res[0])

0.7

In [34]:
actions[np.argmax(res[0])]

'true'

In [35]:
y_test

NameError: name 'y_test' is not defined

In [36]:
actions[np.argmax(y_test[0])]

NameError: name 'y_test' is not defined

# 9. Save Weights

In [37]:
model.save('action.h5')

In [38]:
del model

In [39]:
model.load_weights('action.h5')

NameError: name 'model' is not defined

# 10. Evaluation using Confusion Matrix and Accuracy

In [40]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [41]:
yhat = model.predict(X_test)

NameError: name 'model' is not defined

In [42]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

NameError: name 'y_test' is not defined

In [43]:
plt.figure(figsize=(18,18))

<Figure size 1296x1296 with 0 Axes>

<Figure size 1296x1296 with 0 Axes>

In [44]:
ytrue

NameError: name 'ytrue' is not defined

In [45]:
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'ytrue' is not defined

In [46]:
multilabel_confusion_matrix??

In [47]:
accuracy_score(ytrue, yhat)

NameError: name 'ytrue' is not defined

# 11. Test in Real Time

In [48]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [49]:
def draw_landmarks(image,face_landmarks):
    # Draw face connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec= None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
    
    # Draw face countours (FACEMESH_CONTOURS = frozenset().union(*[FACEMESH_LIPS, FACEMESH_LEFT_EYE, FACEMESH_LEFT_EYEBROW, FACEME) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_contours_style())
    
    # Draw eye iris ( FACEMESH_IRISES = frozenset().union(*[FACEMESH_LEFT_IRIS, FACEMESH_RIGHT_IRIS]) ) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_iris_connections_style())

In [50]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face_mesh)
        print(results)
        
        # Draw landmarks
#         draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_landmarks(image , results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Lie Detection', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

NameError: name 'model' is not defined

In [2]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined